In [3]:
import numpy as np
import pandas as pd
import glob
import os
import shutil

import matplotlib.pyplot as plt
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

In [1]:
!cp /content/drive/MyDrive/model.zip  .

In [4]:
!unzip --q ./model.zip

Archive:  ./model.zip
   creating: saved_model_stop_signals/
  inflating: saved_model_stop_signals/keras_metadata.pb  
   creating: saved_model_stop_signals/variables/
  inflating: saved_model_stop_signals/variables/variables.data-00000-of-00001  
  inflating: saved_model_stop_signals/variables/variables.index  
   creating: saved_model_stop_signals/assets/
  inflating: saved_model_stop_signals/saved_model.pb  


In [5]:
!unzip -qq /content/output_stop_signals.zip

In [ ]:
# !mkdir ./output/0
# !mkdir ./output/1
# !mkdir ./output/2

In [ ]:
train = pd.read_csv('./train.csv', header=None, names=['file', 'cls'])
train.head(5)

,file,cls
0,00001.jpg,no_cars
1,00002.jpg,Off
2,00003.jpg,Off
3,00004.jpg,On
4,00005.jpg,Off


In [ ]:
for i, row in train.iterrows():
  if row['cls'] == 'no_cars':
    shutil.copy('./output/'+ row['file'], './output/0')
  if row['cls'] == 'Off':
    shutil.copy('./output/'+ row['file'], './output/1')
  if row['cls'] == 'On':
    shutil.copy('./output/'+ row['file'], './output/2')

In [ ]:
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
feature_extractor_model = inception_v3

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              21802784  
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 22,065,443
Trainable params: 262,659
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['acc'])

In [ ]:
data_dir = './output'
batch_size = 32
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=batch_size)

class_names = train_ds.class_names
class_names

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

Found 100 files belonging to 3 classes.


In [ ]:
NUM_EPOCHS = 10
history = model.fit(train_ds, epochs=NUM_EPOCHS)

Epoch 1/10
4/4 [==============================] - 0s 92ms/step - loss: 0.3923 - acc: 0.9000
Epoch 2/10
4/4 [==============================] - 0s 52ms/step - loss: 0.8278 - acc: 0.7700
Epoch 3/10
4/4 [==============================] - 0s 53ms/step - loss: 0.4672 - acc: 0.8600
Epoch 4/10
4/4 [==============================] - 0s 68ms/step - loss: 0.1649 - acc: 0.9300
Epoch 5/10
4/4 [==============================] - 0s 54ms/step - loss: 0.2569 - acc: 0.8700
Epoch 6/10
4/4 [==============================] - 0s 54ms/step - loss: 0.1534 - acc: 0.9400
Epoch 7/10
4/4 [==============================] - 0s 55ms/step - loss: 0.0423 - acc: 0.9900
Epoch 8/10
4/4 [==============================] - 0s 58ms/step - loss: 0.0498 - acc: 0.9800
Epoch 9/10
4/4 [==============================] - 0s 73ms/step - loss: 0.0350 - acc: 0.9900
Epoch 10/10
4/4 [==============================] - 0s 56ms/step - loss: 0.0012 - acc: 1.0000


In [ ]:
model.save('./saved_model_stop_signals')
!zip -r model.zip ./saved_model_stop_signals

INFO:tensorflow:Assets written to: ./saved_model_stop_signals/assets


INFO:tensorflow:Assets written to: ./saved_model_stop_signals/assets


  adding: saved_model_stop_signals/ (stored 0%)
  adding: saved_model_stop_signals/keras_metadata.pb (deflated 84%)
  adding: saved_model_stop_signals/variables/ (stored 0%)
  adding: saved_model_stop_signals/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: saved_model_stop_signals/variables/variables.index (deflated 81%)
  adding: saved_model_stop_signals/assets/ (stored 0%)
  adding: saved_model_stop_signals/saved_model.pb (deflated 93%)


In [26]:
import glob
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.models import load_model
from keras.preprocessing import image
from tensorflow import keras

def check_stop_signals(input_dir, output_file="output_color.csv"):
  model = keras.models.load_model('/content/saved_model_stop_signals')
  input_dir = input_dir
  file_list = glob.glob(input_dir+"/*")
  files = []
  preds = []

  for file in file_list:
    file_n = file.replace("\\", "/").split('/')[-1]
    files.append(file_n)

  for file in file_list:
    img = image.load_img(file, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
    pred_c = tf.argmax(pred, axis=1)
    preds.append(pred_c.numpy().ravel())
  
  preds = np.array(preds).ravel()
  output = pd.DataFrame({'file': files, 'pred': preds})
  output['pred'] = output['pred'].replace({0: 'no_cars', 1:'Off', 2:'On'})
  output.to_csv(output_file, index=False, header=False)
  return output